In [1]:
pip install --upgrade snowflake-connector-python pandas

   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.5 MB 5.6 MB/s eta 0:00:02
   ---- ----------------------------------- 1.3/11.5 MB 3.9 MB/s eta 0:00:03
   ------ --------------------------------- 1.8/11.5 MB 3.5 MB/s eta 0:00:03
   --------- ------------------------------ 2.6/11.5 MB 3.4 MB/s eta 0:00:03
   ----------- ---------------------------- 3.4/11.5 MB 3.4 MB/s eta 0:00:03
   ------------- -------------------------- 3.9/11.5 MB 3.4 MB/s eta 0:00:03
   ---------------- ----------------------- 4.7/11.5 MB 3.3 MB/s eta 0:00:03
   ------------------ --------------------- 5.2/11.5 MB 3.3 MB/s eta 0:00:02
   -------------------- ------------------- 5.8/11.5 MB 3.3 MB/s eta 0:00:02
   --------------------- ------------------ 6.3/11.5 MB 3.2 MB/s eta 0:00:02
   ----------------------- ---------------- 6.8/11.5 MB 3.1 MB/s eta 0:00:02
   ------------------------- -------------- 7.3/11.5 MB 3.1 MB/s eta 0:00:02
   ---

In [57]:
import snowflake.connector
import os
import requests
import pandas as pd
import json
PASSWORD = os.getenv('SNOWSQL_PWD')
print(PASSWORD)

5eWxWv4EvyCkhkY


In [169]:
try:
    ctx = snowflake.connector.connect(
        user='ABHINAVSHARMA2002',
        password=PASSWORD,
        account='qraojwa-yg67137'
    )
    cs = ctx.cursor()
    try:
        cs.execute("CREATE WAREHOUSE IF NOT EXISTS tiny_warehouse_mg")
        cs.execute("CREATE DATABASE IF NOT EXISTS testdb_mg")
        cs.execute("USE DATABASE testdb_mg")
        cs.execute("CREATE SCHEMA IF NOT EXISTS testschema_mg")
        cs.execute("USE WAREHOUSE tiny_warehouse_mg")
        cs.execute("USE SCHEMA testschema_mg")
        runQueries(cs)
    except Exception as e:
        print(f"Error: {e}") 
    finally:
        cs.close()
        ctx.close()
except Exception as e:
    print(f"Error: {e}")

                                           author  \
0                                   Anna Washenko   
1                                    Florence Ion   
2                                Brittany Vincent   
3   michael.hicks@futurenet.com (Michael L Hicks)   
4   michael.hicks@futurenet.com (Michael L Hicks)   
..                                            ...   
94                            Bharani Subramaniam   
95                                      Albertini   
96                      alvarorodriguezafiliacion   
97                               Kimberley Snaith   
98                                         AndyKy   

                                                title  \
0   Kaspersky researchers find screenshot-reading ...   
1   The Apple AirPods Pro 3 Might Be Getting Some ...   
2   Amazon Fire TV Cube With All Streaming Service...   
3   Apple is making Samsung look good without even...   
4   Galaxy S25 phones help drive record Qualcomm Q...   
..                   

In [167]:
def runQueries(cs):
    cs.execute(
    """CREATE OR REPLACE TABLE 
    article_table_pandas_1(id INT AUTOINCREMENT PRIMARY KEY, article_name string)""")
    data = fetch_api()
    dictionary = pd.json_normalize(data['articles'])
    df = pd.DataFrame(dictionary)
    print(df)
    # Columns to delete
    columns_to_delete = ["description", "url", "urlToImage", "publishedAt", "content", "source.id",'source.name']

    # Drop the columns
    df = df.drop(columns=columns_to_delete)
    
    query = "INSERT INTO article_table_pandas_1 (%s) VALUES (%s)"
    column_string = ", ".join(df.columns)
    # Convert to a list
    columns_list = column_string.split(", ")    
    valid_column_list = [col for col in columns_list if col in df.columns]
    
    # Insert each row into Snowflake
    """for _, row in df.iterrows():
        params = getColumnValue(valid_column_list,row,column_string)
        print(params)
        cs.execute(
        query, params
        )
        """
    # Fetch results into a Pandas DataFrame
    out = cs.fetch_pandas_all()
    print(out)

In [13]:
def fetch_api():
    url = f"https://newsapi.org/v2/everything?q=apple&from=2025-02-05&to=2025-02-05&sortBy=popularity&apiKey=6725c58e80a8406598187079b42ed0dc"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print("Error fetching exchange rate. Please check your input and try again.")
        return None

In [133]:
def getColumnValue(columns_list,currentRow,columns_string):
    values = [columns_string] + [currentRow[col] for col in columns_list]
    return tuple(values)